In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flowerdatasets/flowers/val/orchid/5479838376_b9ef4b2dd4_k.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/15317253397_c17cf35524_o.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/2068689248_2f31a3160b_k.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/15368996319_9f8354726f_k.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/8505987237_cf07748498_c.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/6467770439_cf417e8003_k.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/2452354536_2e0380c81e_c.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/6803947634_f0ed5fec65_k.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/6921280358_c7a94654e2_o.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/57715059_fc26450350_o.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/12241437214_05d3b7a7eb_o.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/2187054004_ca79f859e6_o.jpg
/kaggle/input/flowerdatasets/flowers/val/orchid/16806723606_297fa28edb_k.jpg
/kaggle/in

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold

# =================================================================
# 1. CẤU HÌNH THAM SỐ CHO FLOWERS DATASET
# =================================================================
# Kích thước ảnh đầu vào - điều chỉnh cho phù hợp với flowers dataset
IMG_HEIGHT = 224
IMG_WIDTH = 224
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
NUM_CLASSES = 7  # flowers có 5 classes

# Tham số ViT - điều chỉnh dựa trên kích thước ảnh mới
PATCH_SIZE = 16
NUM_PATCHES = (IMG_HEIGHT // PATCH_SIZE) * (IMG_WIDTH // PATCH_SIZE)  # 224/16 * 224/16 = 14 * 14 = 196
PATCH_DIM = PATCH_SIZE * PATCH_SIZE * 3  # 16*16*3 = 768
PROJECTION_DIM = 128
NUM_HEADS = 4
TRANSFORMER_LAYERS = 6
MLP_UNITS = [256, 128]

# Tham số training
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

# =================================================================
# 2. ĐỊNH NGHĨA DATA PIPELINE CHO FLOWERS
# =================================================================
def load_and_preprocess_image(filepath, label):
    image = tf.io.read_file(filepath)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image.set_shape([None, None, 3])
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

# =================================================================
# 3. KIẾN TRÚC ViT (GIỮ NGUYÊN VỚI ĐIỀU CHỈNH THAM SỐ)
# =================================================================
# --- Create Patches ---
class PatchLayer(layers.Layer):
    def call(self, images):
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, PATCH_SIZE, PATCH_SIZE, 1],
            strides=[1, PATCH_SIZE, PATCH_SIZE, 1],
            rates=[1, 1, 1, 1],
            padding="VALID"
        )
        patches = tf.reshape(patches, [-1, NUM_PATCHES, PATCH_DIM])
        return patches

# --- Patch Encoder (Linear Projection + Positional Encoding) ---
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.projection = layers.Dense(projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches+1, 
            output_dim=projection_dim
        )

    def call(self, patch_tokens):
        positions = tf.range(start=0, limit=NUM_PATCHES)
        encoded = self.projection(patch_tokens) + self.position_embedding(positions)
        return encoded

def transformer_encoder(inputs):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    # MultiHeadAttention: query/key/value = x
    attn = layers.MultiHeadAttention(
        num_heads=NUM_HEADS, 
        key_dim=PROJECTION_DIM, 
        dropout=0.1
    )(x, x)
    attn = layers.Dropout(0.1)(attn)
    x = layers.Add()([attn, inputs])

    # FFN
    y = layers.LayerNormalization(epsilon=1e-6)(x)
    y = layers.Dense(MLP_UNITS[0], activation='gelu')(y)
    y = layers.Dropout(0.1)(y)
    y = layers.Dense(PROJECTION_DIM)(y)
    return layers.Add()([x, y])

# --- Build ViT model ---
def build_vit(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES):
    inputs = layers.Input(shape=input_shape)

    # 1) Make patches
    patches = PatchLayer()(inputs)  # -> (batch, NUM_PATCHES, PATCH_DIM)

    # 2) Patch encoding (linear proj + positional embedding)
    patch_embeddings = PatchEncoder(NUM_PATCHES, PROJECTION_DIM)(patches)

    # 3) class token variable (trainable)
    class_token = tf.Variable(
        tf.zeros((1, 1, PROJECTION_DIM)), 
        trainable=True, 
        name="class_token"
    )

    # 4) use a Lambda layer to repeat & concat class token
    def _prepend_token(patch_emb):
        batch = tf.shape(patch_emb)[0]
        tokens = tf.repeat(class_token, repeats=batch, axis=0)
        return tf.concat([tokens, patch_emb], axis=1)

    x = layers.Lambda(_prepend_token, name="prepend_class_token")(patch_embeddings)

    # 5) Transformer encoder stacks
    for i in range(TRANSFORMER_LAYERS):
        x = transformer_encoder(x)

    # 6) Take class token output (index 0)
    x = layers.LayerNormalization(epsilon=1e-6, name="pre_head_ln")(x[:, 0])

    # 7) MLP head
    x = layers.Dense(128, activation="gelu")(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="predictions")(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="ViT_Flowers")

# =================================================================
# 4. CHUẨN BỊ DỮ LIỆU FLOWERS
# =================================================================
def load_image_paths_labels(data_dir):
    """Tải đường dẫn ảnh và labels từ thư mục"""
    class_names = sorted(os.listdir(data_dir))
    paths = []
    labels = []
    for i, cls in enumerate(class_names):
        cls_dir = os.path.join(data_dir, cls)
        for fname in os.listdir(cls_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                paths.append(os.path.join(cls_dir, fname))
                labels.append(i)
    return paths, labels, class_names

# Tải dữ liệu
train_dir = '/kaggle/input/flowerdatasets/flowers/train'
val_dir = '/kaggle/input/flowerdatasets/flowers/val'
test_dir = '/kaggle/input/flowerdatasets/flowers/test'

train_paths, train_labels, class_names = load_image_paths_labels(train_dir)
val_paths, val_labels, _ = load_image_paths_labels(val_dir)
test_paths, test_labels, _ = load_image_paths_labels(test_dir)

print(f"Classes: {class_names}")
print(f"Train samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Test samples: {len(test_paths)}")

# Chuyển labels sang one-hot encoding
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, num_classes=NUM_CLASSES)
val_labels_one_hot = tf.keras.utils.to_categorical(val_labels, num_classes=NUM_CLASSES)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels, num_classes=NUM_CLASSES)

# =================================================================
# 5. TẠO DATA PIPELINE
# =================================================================
# Tạo train dataset với augmentation
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels_one_hot))
train_ds = train_ds.shuffle(len(train_paths)) \
                 .map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE) \
                 .map(lambda x, y: (data_augmentation(x, training=True), y), 
                      num_parallel_calls=tf.data.AUTOTUNE) \
                 .batch(BATCH_SIZE) \
                 .prefetch(tf.data.AUTOTUNE)

# Tạo validation dataset (không augmentation)
val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels_one_hot))
val_ds = val_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE) \
               .batch(BATCH_SIZE) \
               .prefetch(tf.data.AUTOTUNE)

# Tạo test dataset (không augmentation)
test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels_one_hot))
test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE) \
               .batch(BATCH_SIZE) \
               .prefetch(tf.data.AUTOTUNE)

# =================================================================
# 6. HUẤN LUYỆN MÔ HÌNH
# =================================================================
# Tạo mô hình
model = build_vit()

# Compile mô hình
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-7
    )
]

# Huấn luyện
print("Bắt đầu huấn luyện ViT trên Flowers dataset...")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks,
    verbose=1
)

# =================================================================
# 7. ĐÁNH GIÁ MÔ HÌNH
# =================================================================
print("\nĐánh giá trên tập validation:")
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

print("\nĐánh giá trên tập test:")
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# =================================================================
# 8. LƯU MÔ HÌNH
# =================================================================
# Lưu trọng số mô hình
model.save_weights('/kaggle/working/vit_flowers_model.weights.h5')

# Lưu lịch sử huấn luyện
np.save('/kaggle/working/vit_flowers_history.npy', history.history)

print("Huấn luyện hoàn tất và mô hình đã được lưu!")

2025-11-25 14:05:28.783764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764079528.954403      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764079529.006602      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Classes: ['daisy', 'dandelion', 'lily', 'orchid', 'rose', 'sunflower', 'tulip']
Train samples: 8069
Validation samples: 2306
Test samples: 1156
Bắt đầu huấn luyện ViT trên Flowers dataset...
Epoch 1/50


I0000 00:00:1764079562.053087      62 service.cc:148] XLA service 0x79c27c0037a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764079562.054130      62 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1764079564.678759      62 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/253 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - accuracy: 0.1545 - loss: 2.5982

I0000 00:00:1764079578.652384      62 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


253/253 ━━━━━━━━━━━━━━━━━━━━ 103s 276ms/step - accuracy: 0.3289 - loss: 1.7033 - val_accuracy: 0.6769 - val_loss: 0.9023 - learning_rate: 0.0010
Epoch 2/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 49s 195ms/step - accuracy: 0.6741 - loss: 0.9097 - val_accuracy: 0.7212 - val_loss: 0.7592 - learning_rate: 0.0010
Epoch 3/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 49s 194ms/step - accuracy: 0.7049 - loss: 0.7973 - val_accuracy: 0.7259 - val_loss: 0.7922 - learning_rate: 0.0010
Epoch 4/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 50s 196ms/step - accuracy: 0.7206 - loss: 0.7601 - val_accuracy: 0.7173 - val_loss: 0.7764 - learning_rate: 0.0010
Epoch 5/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 49s 193ms/step - accuracy: 0.7427 - loss: 0.7186 - val_accuracy: 0.7602 - val_loss: 0.6546 - learning_rate: 0.0010
Epoch 6/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 49s 193ms/step - accuracy: 0.7448 - loss: 0.6958 - val_accuracy: 0.7658 - val_loss: 0.6131 - learning_rate: 0.0010
Epoch 7/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 49s 192ms/step - accuracy: 0.7718 - loss